In [2]:
from eBallena import eBallena
from Data import FSDD
from Baseline import generateReservoir, generateTopologyInput, assignWeightsReservoir, assignWeightsInputs
from Utils import evalKfold, Parallel

import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
# ==============
#  LOAD DATASET
# ==============

# FSDD(time_bin=80, max_spikes=10, n_dim=26,  hop_lenght=400).preProcessAndSave(random_state=12)
fsdd = FSDD().load()

In [ ]:
# ==============
#   GET METRICS
#   FUNCTION
# ==============

def get_metrics(n_neu, c_ie, L, tau, w_in, w_re, pneu_e, n_processes=4):

	# =================
	# GENERAR TOPOLOGIA
	connections_in, syn_type_in = generateTopologyInput(n_inputs=26,n_neu_reservoir=n_neu,
												  		prob_connection=0.03)
	connections_re, neuron_type_re = generateReservoir(num_neuron=n_neu, prob_exitatory=pneu_e,  L=L, c_ie=c_ie)

	syn_in = assignWeightsInputs(connections_in, syn_type_in, w_in)
	syn_re = assignWeightsReservoir(connections_re, neuron_type_re, w_re)

	# ==============
	# CONFIGURACION
	# DE SIMULACION
	config = eBallena.createConfig(n_inputs=26, n_neu=n_neu, tau=tau, max_sim_time=3_000, 
									refractory_time=5, threshold=-55)
		
	simulation = Parallel(lambda instance: eBallena.simulate(config,instance,syn_in, syn_re, mode='STATE'), n_processes=n_processes)

	# =================
	#     SIMULATE
	s = []
	t = []
	for fold in fsdd.getFolds():
		s.append( np.array(simulation.evaluateY(fold['d'])) )
		t.append( fold['t'] )	
	# =================
	#   PARSE RESULTS
	get_acc = Parallel(lambda test_idx:evalKfold(s, t, test_idx),n_processes=n_processes)
	all_acc = np.array(get_acc.evaluateY([i for i in range(5)]))

	acc , acc_train  = all_acc.mean(axis=0).round(3)
	mean, mean_train = all_acc.std(axis=0).round(3)
	num_spikes       = np.concatenate(s).sum(axis=1)

	return acc, acc_train, mean, mean_train, num_spikes.mean().round(),num_spikes.std().round()


In [ ]:
# ==============
#  ALL CONFS
# ==============

all_neu = [125,512, 1000]
all_cie = [0.05, 0.1, 0.2, 0.3, 0.4]
all_l   = [2,3]
all_pex = [0.7, 0.75, 0.8, 0.85, 0.9]
all_tau = [ 10, 20, 30, 80, 120,200]

all_configurations = []
for neu in all_neu:
	for c_ie in all_cie:
		for L in all_l:
			for pneu_e in all_pex:
				for tau in all_tau:
					for _ in range(5):
						all_configurations.append( (neu,c_ie,L,pneu_e,tau) )

In [ ]:
# ==============
#  GET RESULTS
# ==============

n_processes = 32
results 	= []
for conf in tqdm(all_configurations):
	neu,c_ie,L,pneu_e,tau = conf
	metrics = get_metrics(n_neu=neu, c_ie=c_ie, L=L, tau=tau , w_in=5 ,w_re=2.1 , pneu_e=pneu_e, n_processes=n_processes)
	results.append(metrics)

with open('results.pickle','wb') as file:
	pickle.dump( {'all_configuracions':all_configurations, 'results':results}, file )